In [1]:
import pandas as pd
import utils
import numpy as np
import warnings
from tqdm import tqdm
from copairs.map import average_precision, mean_average_precision
import logging

logging.basicConfig(format="%(levelname)s:%(asctime)s:%(name)s:%(message)s")
logging.getLogger("copairs").setLevel(logging.INFO)

warnings.simplefilter(action="ignore", category=FutureWarning)

In [2]:
operations = "wellpos_cc_var_mad_outlier_featselect_sphering_harmony"
batch_size = 20000
null_size = 20000
fdr = 0.1

### Prepare the data

#### Read the ORF parquet file

In [3]:
orf_df = pd.read_parquet(
    f"../profiles/profiles_{operations}.parquet"
)
orf_df.shape

(81660, 726)

#### Add annotations

In [4]:
orf_metdata_df = pd.read_csv(
    "../00.download-and-process-annotations/output/orf_metadata.tsv.gz", sep="\t"
)
compound_metadata_df = pd.read_csv(
    "../datasets/metadata/compound.csv.gz", usecols=["Metadata_JCP2022"]
).assign(
    Metadata_pert_type=lambda x: np.where(
        x["Metadata_JCP2022"] == "JCP2022_999999", "empty", "poscon"
    )
)

metadata_df = pd.concat(
    [
        orf_metdata_df,
        compound_metadata_df
    ],
    join="outer",
    ignore_index=True,
)

orf_df = orf_df.merge(metadata_df, on="Metadata_JCP2022", how="inner")
orf_df.shape

(81660, 750)

#### Remove empty wells

In [5]:
orf_df = utils.remove_empty_wells(orf_df)
orf_df.shape

(81490, 750)

#### Remove `poscon` wells.

In [6]:
orf_df = orf_df.query('Metadata_pert_type!="poscon"').reset_index(drop=True)
orf_df.shape

(79560, 750)

#### Remove `BAD CONSTRUCT` profiles

In [7]:
orf_df = orf_df.query('Metadata_broad_sample!="BAD CONSTRUCT"').reset_index(drop=True)
orf_df.shape

(79560, 750)

#### Remove featues with `nan` values.
These need to be removed as the `nan` values will cause the mean average precision calculation to fail.

In [8]:
orf_df = utils.remove_nan_features(orf_df)

Removed nan features: []


#### Remove low infection effiency wells

In [9]:
# Add platemap name

platemap_df = pd.read_csv(
    "../00.download-and-process-annotations/input/experiment-metadata.tsv",
    sep="\t",
    usecols=["Plate_Map_Name", "Assay_Plate_Barcode"],
).rename(
    columns={
        "Plate_Map_Name": "Metadata_plate_map_name",
        "Assay_Plate_Barcode": "Metadata_Plate",
    }
)

orf_df = orf_df.merge(platemap_df, on="Metadata_Plate", how="left")

orf_df = utils.remove_low_infection_efficiency_wells(orf_df)
orf_df.shape

(72343, 751)

### Calculate mAP for each ORF perturbation

In [10]:
# Adding a new column for negative control
orf_df["Metadata_negcon"] = np.where(orf_df["Metadata_pert_type"] == "negcon", 1, 0)

In [11]:
pos_sameby = ["Metadata_JCP2022"]
pos_diffby = []
neg_sameby = ["Metadata_Plate"]
neg_diffby = ["Metadata_negcon"]

In [12]:
metadata_df = utils.get_metadata(orf_df)
feature_df = utils.get_featuredata(orf_df)
feature_values = feature_df.values

In [13]:
result = average_precision(
    metadata_df, feature_values, pos_sameby, pos_diffby, neg_sameby, neg_diffby, batch_size=batch_size
)

INFO:2024-05-01 15:38:01,184:copairs:Indexing metadata...
INFO:2024-05-01 15:38:01,262:copairs:Finding positive pairs...
INFO:2024-05-01 15:38:01,932:copairs:Finding negative pairs...
INFO:2024-05-01 15:38:02,715:copairs:Computing positive similarities...


  0%|          | 0/76 [00:00<?, ?it/s]

INFO:2024-05-01 15:38:05,966:copairs:Computing negative similarities...


  0%|          | 0/52 [00:00<?, ?it/s]

INFO:2024-05-01 15:38:08,429:copairs:Building rank lists...
INFO:2024-05-01 15:38:10,019:copairs:Computing average precision...
INFO:2024-05-01 15:38:10,087:copairs:Creating result DataFrame...
INFO:2024-05-01 15:38:10,091:copairs:Finished.


In [14]:
# Remove negcon
result = result.query('Metadata_pert_type!="negcon"').reset_index(drop=True)

In [15]:
agg_result = (
    mean_average_precision(result, pos_sameby, null_size=null_size, threshold=fdr, seed=12527)
    .rename(columns={'average_precision': 'mean_average_precision'})
)

INFO:2024-05-01 15:38:10,263:copairs:Computing null_dist...


  0%|          | 0/18 [00:00<?, ?it/s]

INFO:2024-05-01 15:38:10,315:copairs:Computing p-values...


  0%|          | 0/13739 [00:00<?, ?it/s]

In [16]:
agg_result.to_csv(f"output/phenotypic-activity-{operations}.csv.gz", index=False)